In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

In [51]:
subject_id = '23'
data = pd.read_csv('sub-23/sub-23_behavioral/23_vm_30xx_2022_Apr_29_1531.csv')

In [52]:
data = data[data['target_onset'].notna()]

In [53]:
data['trial_type'] = 'go'
data['trial_type'][data['go'] == 0 ] = 'nogo'

/tmp/ipykernel_28066/1044001338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['trial_type'][data['go'] == 0 ] = 'nogo'


In [54]:
#Since exp restarted for sub-01, remapping the original loops to what they would've been
#Run only for the full file, not individual block files
#data['alternate_baseline_rotation.thisTrialN'] = data['alternate_baseline_rotation.thisTrialN'] + 1

In [55]:
def save_events_file(subject_id, chosen_block, run_no):
    prep_duration = chosen_block.signal_onset - chosen_block.target_onset
    exec_time = chosen_block.feedback_onset - chosen_block.signal_onset
    wait_time = chosen_block.target_onset - chosen_block.wait_for_target_onset
    event_trial_type = chosen_block.trial_type + '_' + chosen_block.target_angle.astype(int).astype(str)
    exec_type = list()
    for typ, angle in zip(chosen_block.trial_type, list(chosen_block.target_angle.astype(int).astype(str))):
        if typ == 'go':
            e = 'movement_' + angle
            exec_type.append(e)
        else:
            e = 'non_movement_' + angle
            exec_type.append(e)
    prep_event_trial_type = event_trial_type +  str('_prep')
    wait_event_trial_type = np.repeat(str('wait'), len(event_trial_type))
    
    onset = list(chosen_block.wait_for_target_onset) + list(chosen_block.target_onset) + list(chosen_block.signal_onset)
    trial_type = list(wait_event_trial_type) + list(prep_event_trial_type) + exec_type
    duration = list(wait_time) + list(prep_duration) + list(exec_time)
    
    #print (prep_duration, prep_event_trial_type)
    events = pd.DataFrame({'onset': onset, 
                       'trial_type': trial_type,
                       'duration': duration                      
                      })
    dir_name = 'sub-' + subject_id + '/run-' + run_no 
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    file_name = dir_name + '/events.csv'
    events.to_csv(file_name)
    #return events

In [19]:
#double check the run numbers
for i in np.unique(data['alternate_baseline_rotation.thisTrialN']):
    temp_data = data[data['alternate_baseline_rotation.thisTrialN'] == i]
    

    
    baseline_block = temp_data[temp_data['baseline_inner_circle.started'].notna()]
    ## Setting signal time == actual signal time + IT
    baseline_block.loc[baseline_block.trial_type == 'go', 'signal_onset'] = baseline_block.loc[baseline_block.trial_type == 'go', 'signal_onset'] + baseline_block[baseline_block['trial_type'] == 'go']['IT'].fillna(0)

    save_events_file(subject_id, baseline_block, str(int(i)*2+1).zfill(2))


    rotation_block = temp_data[temp_data['rotation_inner_circle.started'].notna()]
    ITs = rotation_block[rotation_block['trial_type'] == 'go']['IT'].values

    ## Setting signal time == actual signal time + IT
    rotation_block.loc[rotation_block.trial_type == 'go', 'signal_onset'] = rotation_block.loc[rotation_block.trial_type == 'go', 'signal_onset'] + rotation_block[rotation_block['trial_type'] == 'go']['IT'].fillna(0)
    
    save_events_file(subject_id, rotation_block, str(int(i)*2+2).zfill(2))


/home/vm01/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/vm01/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/vm01/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [57]:
temp_data = data[data['alternate_baseline_rotation.thisTrialN'] == 0]
baseline_block = temp_data[temp_data['baseline_inner_circle.started'].notna()].reset_index()
# x_end, y_end = (json.loads(baseline_block['go_mouse.x'][5])[-1], json.loads(baseline_block['go_mouse.y'][5])[-1])

In [75]:
json.loads(baseline_block['go_mouse.x'][3])[-1]

-0.096354164

In [73]:
baseline_block['target_angle'][4]

45.0

In [ ]:
(0.234375, 0.0703125)

In [ ]:
#Read a straight-rotate block pair in a temporary variable
temp_data = data[data['alternate_baseline_rotation.thisTrialN'] == i]

#Extract the baseline block from the block pair. Done by selecting those rows for which 
baseline_block = temp_data[temp_data['baseline_inner_circle.started'].notna()]
baseline_block.loc[baseline_block.trial_type == 'go', 'signal_onset'] = baseline_block.loc[baseline_block.trial_type == 'go', 'signal_onset'] + baseline_block[baseline_block['trial_type'] == 'go']['IT'].fillna(0)

# save_events_file(subject_id, baseline_block, str(int(i)*2+1).zfill(2))



prep_duration = chosen_block.signal_onset - chosen_block.target_onset
exec_time = chosen_block.feedback_onset - chosen_block.signal_onset
wait_time = chosen_block.target_onset - chosen_block.wait_for_target_onset
event_trial_type = chosen_block.trial_type + '_' + chosen_block.target_angle.astype(int).astype(str)
exec_type = list()
for typ, angle in zip(chosen_block.trial_type, list(chosen_block.target_angle.astype(int).astype(str))):
    if typ == 'go':
        e = 'movement_' + angle
        exec_type.append(e)
    else:
        e = 'non_movement_' + angle
        exec_type.append(e)
prep_event_trial_type = event_trial_type +  str('_prep')
wait_event_trial_type = np.repeat(str('wait'), len(event_trial_type))

onset = list(chosen_block.wait_for_target_onset) + list(chosen_block.target_onset) + list(chosen_block.signal_onset)
trial_type = list(wait_event_trial_type) + list(prep_event_trial_type) + exec_type
duration = list(wait_time) + list(prep_duration) + list(exec_time)

#print (prep_duration, prep_event_trial_type)
events = pd.DataFrame({'onset': onset, 
                   'trial_type': trial_type,
                   'duration': duration                      
                  })
dir_name = 'sub-' + subject_id + '/run-' + run_no 
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
file_name = dir_name + '/events.csv'
events.to_csv(file_name)


In [253]:
events = save_events_file(rotation_block, '1')

In [308]:
rotation_block[rotation_block['trial_type'] == 'go']['signal_onset']

3      23.713176
6      31.787733
9      43.075715
18     82.603426
25    115.963152
28    130.632421
33    146.053303
38    163.999711
47    202.911588
50    209.700720
55    224.389177
58    233.892747
61    241.363289
74    305.895287
79    328.668915
82    338.334487
87    360.947555
90    367.265466
93    373.877938
96    380.975361
Name: signal_onset, dtype: float64

In [288]:
rotation_block[rotation_block['trial_type'] == 'go']['IT']  

3          NaN
6     2.229609
9     0.638783
18    0.595668
25    0.605372
28    0.698882
33    0.511706
38    0.514763
47    0.738032
50    0.611868
55    0.592949
58    0.524447
61    0.567836
74    0.496074
79    0.695878
82    0.555495
87    0.579989
90    0.826583
93    0.474808
96    1.034435
Name: IT, dtype: float64

## Use events files to extract trial level events. 

In [8]:
subject_id = '01'
for run in range(1, 11):
    events_file = 'sub-' + subject_id + '/run-' + str(run).zfill(2) + '/events.csv'
    events = pd.read_csv(events_file).drop('Unnamed: 0', 1)

In [18]:
events

,onset,trial_type,duration
0,1.354091,wait,2.018276
1,8.063258,wait,2.012760
2,16.198916,wait,3.012424
3,23.677944,wait,3.024296
4,33.009063,wait,5.012558
...,...,...,...
115,346.454803,movement_207,0.590417
116,353.586614,non_movement_153,3.068839
117,364.310611,non_movement_153,0.382517
118,373.115227,non_movement_207,3.075370


In [20]:
def save_prep_events(alternate_loop_no, baseline_run_no, rotation_run_no):
    d1 = data[data['alternate_baseline_rotation.thisN'] == alternate_loop_no]
    baseline_idx = np.unique(d1['baseline_loop.thisTrialN'], return_index=True)[1][:40]
    rotation_idx = np.unique(d1['rotation_loop.thisTrialN'], return_index=True)[1][:40]
    
    #For baseline Runs
    target_directions = np.array(d1.iloc[baseline_idx]['target_angle']).astype(int).astype(str)
    go_trial = np.array(d1.iloc[baseline_idx]['go']).astype(str)
    #iti = np.array(d1.iloc[baseline_idx]['ITI'])
    #it = np.array(d1.IT[baseline_idx])

    baseline_go_prep_time = d1.iloc[baseline_idx][d1.go == 1.0]['baseline_inner_circle_go.started'].values - d1.iloc[baseline_idx][d1.go == 1.0]['target.started'].values
    baseline_nogo_prep_time = d1.iloc[baseline_idx][d1.go == 0.0]['baseline_inner_circle_nog.started'].values - d1.iloc[baseline_idx][d1.go == 0.0]['target.started'].values

    target_onset = np.array(d1.iloc[baseline_idx]['target.started'] - np.array(d1.iloc[baseline_idx]['target.started'])[0])

    duration = np.zeros(40)
    duration[go_trial == '1.0'] = baseline_go_prep_time
    duration[go_trial == '0.0'] = baseline_nogo_prep_time
    go_trial[go_trial == '1.0'] = 'go_'
    go_trial[go_trial == '0.0'] = 'nogo_'
    
    ##Saving file
    events = pd.DataFrame({'onset': target_onset.astype(float),
                           'trial_type':np.add(go_trial.astype(object), target_directions.astype(object)), 
                           'duration': duration                      
                          })
    events_non_prep_onsets = events['onset'].astype(float) + events['duration'].astype(float)
    events_non_prep_durations = np.array(events['onset'][1:]) - np.array(events_non_prep_onsets[:-1])
    non_prep_events = pd.DataFrame({'onset': events_non_prep_onsets[1:],
                                    'trial_type': 'non_prep_'+ events['trial_type'][1:],
                                    'duration': events_non_prep_durations
                                   })
    events = pd.concat([events, non_prep_events])

    sorted_events = events.sort_values(by = 'trial_type')
    baseline_file_name = 'events/prep_time_events/prep_events_run_' + str(baseline_run_no) + '.csv'
#     sorted_events.to_csv(baseline_file_name)
    
    #For rotation Runs
    target_directions = np.array(d1.iloc[rotation_idx]['target_angle']).astype(int).astype(str)
    go_trial = np.array(d1.iloc[rotation_idx]['go']).astype(str)
    #iti = np.array(d1.iloc[rotation_idx]['ITI'])
    #it = np.array(d1.IT[rotation_idx])
    non_prep_trial = np.repeat(['non_prep'], len(go_trial))

    rotation_go_prep_time = d1.iloc[rotation_idx][d1.go == 1.0]['rotation_inner_circle_go.started'].values - d1.iloc[rotation_idx][d1.go == 1.0]['target_rotated.started'].values
    rotation_nogo_prep_time = d1.iloc[rotation_idx][d1.go == 0.0]['rotation_inner_circle_nogo.started'].values - d1.iloc[rotation_idx][d1.go == 0.0]['target_rotated.started'].values

    target_onset = np.array(d1.iloc[rotation_idx]['target_rotated.started'] - np.array(d1.iloc[rotation_idx]['target_rotated.started'])[0])

    duration = np.zeros(40)
    duration[go_trial == '1.0'] = rotation_go_prep_time
    duration[go_trial == '0.0'] = rotation_nogo_prep_time
    go_trial[go_trial == '1.0'] = 'go_'
    go_trial[go_trial == '0.0'] = 'nogo_'

    events = pd.DataFrame({'onset': target_onset.astype(float),
                           'trial_type':np.add(go_trial.astype(object), target_directions.astype(object)), 
                           'duration': duration                      
                          })
    events_non_prep_onsets = events['onset'].astype(float) + events['duration'].astype(float)
    events_non_prep_durations = np.array(events['onset'][1:]) - np.array(events_non_prep_onsets[:-1])
    non_prep_events = pd.DataFrame({'onset': events_non_prep_onsets[1:],
                                    'trial_type': 'non_prep_'+ events['trial_type'][1:],
                                    'duration': events_non_prep_durations
                                   })
    events = pd.concat([events, non_prep_events])
    sorted_events = events.sort_values(by = 'onset')
    rot_file_name = 'events/prep_time_events/prep_events_run_' + str(rotation_run_no) + '.csv'

#     sorted_events.to_csv(rot_file_name)
    return sorted_events, events_non_prep_onsets, events_non_prep_durations

In [29]:
events, events_non_prep_onsets, dur= save_prep_events(1.0, 2, 3)

<ipython-input-20-ea1773784845>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_go_prep_time = d1.iloc[baseline_idx][d1.go == 1.0]['baseline_inner_circle_go.started'].values - d1.iloc[baseline_idx][d1.go == 1.0]['target.started'].values
<ipython-input-20-ea1773784845>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_go_prep_time = d1.iloc[baseline_idx][d1.go == 1.0]['baseline_inner_circle_go.started'].values - d1.iloc[baseline_idx][d1.go == 1.0]['target.started'].values
<ipython-input-20-ea1773784845>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_nogo_prep_time = d1.iloc[baseline_idx][d1.go == 0.0]['baseline_inner_circle_nog.started'].values - d1.iloc[baseline_idx][d1.go == 0.0]['target.started'].values
<ipython-input-20-ea1773784845>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  baseline_nogo_prep_time = d1.iloc[baseline_

In [30]:
events

,onset,trial_type,duration
0,0.000000,nogo_99,2.044656
1,8.036178,go_207,2.222336
1,10.258514,non_prep_go_207,5.991522
2,15.750641,go_153,1.874295
2,17.624936,non_prep_go_153,5.492127
...,...,...,...
37,368.429258,non_prep_nogo_153,12.426512
38,375.305069,go_153,1.907661
38,377.212730,non_prep_go_153,6.875812
39,382.688078,go_99,2.070210


In [467]:
def save_pre_prep_events(alternate_loop_no, baseline_run_no, rotation_run_no):
    d1 = data[data['alternate_baseline_rotation.thisN'] == alternate_loop_no]
    baseline_idx = np.unique(d1['baseline_loop.thisTrialN'], return_index=True)[1][:40]
    rotation_idx = np.unique(d1['rotation_loop.thisTrialN'], return_index=True)[1][:40]
    
    #For baseline Runs
    target_directions = np.array(d1.iloc[baseline_idx]['target_angle']).astype(int).astype(str)
    go_trial = np.array(d1.iloc[baseline_idx]['go']).astype(str)
    #iti = np.array(d1.iloc[baseline_idx]['ITI'])
    #it = np.array(d1.IT[baseline_idx])

    baseline_go_prep_time = d1.iloc[baseline_idx][d1.go == 1.0]['baseline_inner_circle_go.started'].values - d1.iloc[baseline_idx][d1.go == 1.0]['target.started'].values
    baseline_nogo_prep_time = d1.iloc[baseline_idx][d1.go == 0.0]['baseline_inner_circle_nog.started'].values - d1.iloc[baseline_idx][d1.go == 0.0]['target.started'].values

    target_onset = np.array(d1.iloc[baseline_idx]['target.started'] - np.array(d1.iloc[baseline_idx]['target.started'])[0])

    duration = np.zeros(40)
    duration[go_trial == '1.0'] = baseline_go_prep_time
    duration[go_trial == '0.0'] = baseline_nogo_prep_time
    go_trial[go_trial == '1.0'] = 'go_'
    go_trial[go_trial == '0.0'] = 'nogo_'
    
    ##Saving file
    events = pd.DataFrame({'onset': target_onset.astype(str),
                           'trial_type':np.add(go_trial.astype(object), target_directions.astype(object)), 
                           'duration': duration                      
                          })
    sorted_events = events.sort_values(by = 'trial_type')
    baseline_file_name = 'prep_events_run_' + str(baseline_run_no) + '.csv'
    sorted_events.to_csv(baseline_file_name)
    
    #For rotation Runs
    target_directions = np.array(d1.iloc[rotation_idx]['target_angle']).astype(int).astype(str)
    go_trial = np.array(d1.iloc[rotation_idx]['go']).astype(str)
    #iti = np.array(d1.iloc[rotation_idx]['ITI'])
    #it = np.array(d1.IT[rotation_idx])

    rotation_go_prep_time = d1.iloc[rotation_idx][d1.go == 1.0]['rotation_inner_circle_go.started'].values - d1.iloc[rotation_idx][d1.go == 1.0]['target_rotated.started'].values
    rotation_nogo_prep_time = d1.iloc[rotation_idx][d1.go == 0.0]['rotation_inner_circle_nogo.started'].values - d1.iloc[rotation_idx][d1.go == 0.0]['target_rotated.started'].values

    target_onset = np.array(d1.iloc[rotation_idx]['target_rotated.started'] - np.array(d1.iloc[rotation_idx]['target_rotated.started'])[0])

    duration = np.zeros(40)
    duration[go_trial == '1.0'] = rotation_go_prep_time
    duration[go_trial == '0.0'] = rotation_nogo_prep_time
    go_trial[go_trial == '1.0'] = 'go_'
    go_trial[go_trial == '0.0'] = 'nogo_'

    events = pd.DataFrame({'onset': target_onset.astype(str),
                           'trial_type':np.add(go_trial.astype(object), target_directions.astype(object)), 
                           'duration': duration                      
                          })
    sorted_events = events.sort_values(by = 'trial_type')
    rot_file_name = 'prep_events_run_' + str(rotation_run_no) + '.csv'

    sorted_events.to_csv(rot_file_name)

<ipython-input-468-bdd161078442>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rotation_go_prep_time = d1.iloc[rotation_idx][d1.go == 1.0]['rotation_inner_circle_go.started'].values - d1.iloc[rotation_idx][d1.go == 1.0]['target_rotated.started'].values
<ipython-input-468-bdd161078442>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rotation_go_prep_time = d1.iloc[rotation_idx][d1.go == 1.0]['rotation_inner_circle_go.started'].values - d1.iloc[rotation_idx][d1.go == 1.0]['target_rotated.started'].values
<ipython-input-468-bdd161078442>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rotation_nogo_prep_time = d1.iloc[rotation_idx][d1.go == 0.0]['rotation_inner_circle_nogo.started'].values - d1.iloc[rotation_idx][d1.go == 0.0]['target_rotated.started'].values
<ipython-input-468-bdd161078442>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rotation_nogo_prep

In [313]:
# For non-prep time
duration = target_onset[1:] - target_onset[:-1]

In [ ]:
# Last 2 trials were omitted cause scan finished first
duration = duration[:-2]
target_onset = target_onset[:-2]
go_trial = go_trial[:-2]
target_directions = target_directions[:-2]
go_trial[np.where(go_trial == '1.0')[0]] = 'go_'
go_trial[np.where(go_trial == '0.0')[0]] = 'nogo_'
